<a href="https://colab.research.google.com/github/Starrysmh/MARK5828-AdvertisingAnalytics-Project/blob/main/7_1_Interaction_Term_for_Combination_of_2_Main_Variables_signalchain_person%26instru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd 
import numpy as np
import os
import statsmodels.api as sm
from week2_helpers import variance_inflation_factor

In [12]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns; sns.set()
from google.colab import files
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [13]:
print('Please upload the week2_helper python file')
files.upload()

Please upload the week2_helper python file


Saving week2_helpers.py to week2_helpers.py


{'week2_helpers.py': b'"""\r\nSpecial Utility Functions for MARK5828 Week 2 Tutorial Content. The functions saved here are unnecessary for the students to learn, but they can read if they are curious.\r\nAuthor: James Lin\r\nSpecial Thanks: Daniel-Han Chan\r\n"""\r\n\r\nimport itertools\r\nfrom scipy.linalg.blas import ssyrk\r\nfrom scipy.linalg.lapack import spotrf, sposv\r\nimport numpy as np\r\nimport pandas as pd\r\n\r\nimport urllib.parse\r\nimport requests\r\n\r\ndef convert_to_seconds(time_str):\r\n    h, m, s = time_str.split(":")\r\n    return int(h) * 3600 + int(m) * 60 + round(float(s))\r\n\r\ndef get_source_language_v2(data):\r\n    return data["videos"][0]["insights"]["sourceLanguage"]\r\n\r\ndef get_video_duration_v2(data):\r\n    return data["summarizedInsights"]["duration"]["seconds"]\r\n\r\ndef get_faces_v2(data):\r\n    face_names = []\r\n    face_duration = []\r\n\r\n    for i, face in enumerate(data["summarizedInsights"]["faces"]):\r\n        face_names.append(face[

In [14]:
from week2_helpers import variance_inflation_factor

In [15]:
def file_reader():
  
  uploaded_file=files.upload()
  uploaded_file=list(uploaded_file.keys())[0]
  if uploaded_file.endswith('csv'):
    data=pd.read_csv(uploaded_file)
  else:
    data=pd.read_excel(uploaded_file)
  return data

In [16]:
print('Please upload the merged data')
merged_data=file_reader()

Please upload the merged data


Saving regression_data_signalchain.csv to regression_data_signalchain.csv


In [32]:
dummy_data = merged_data[["likeCount","yesPerson","YesFinalInstrument"]]

In [33]:
dummy_data

,likeCount,yesPerson,YesFinalInstrument
0,90,0,0
1,75,0,0
2,48,0,0
3,28,0,1
4,58,0,0
...,...,...,...
2032,226,0,1
2033,228,0,1
2034,237,0,1
2035,258,0,1


#H1: with person in the post incrreases social media ad popularity. 
#H2: with instrument in the post decreases social media ad popularity. 


In [34]:
Y=dummy_data['likeCount']
X=dummy_data.drop(['likeCount'],axis=1).astype(float)
X=sm.add_constant(X)
model=sm.OLS(Y,X)
results=model.fit()

In [35]:
vif = variance_inflation_factor(X)
vif_store=vif.drop('const',axis=0)
vif_store.sort_values('vif',ascending=False)
df_coeffs = pd.DataFrame({"Coefficients": results.params, "p": results.pvalues, "vif": vif["vif"]})
if 'const' in df_coeffs.index:
    df_coeffs = df_coeffs.drop("const", axis=0)
df_coeffs_sorted = df_coeffs.reindex(df_coeffs["p"].sort_values(ascending=True).index)

In [36]:
df_coeffs_sorted 

,Coefficients,p,vif
YesFinalInstrument,28.289326,2.393280e-09,1.010037
yesPerson,-14.902132,6.609719e-03,1.010037


in this regression result, we can see the effect of present of person and instrument on number of likes

# Approach 1 - make interaction variable X3 = X1 x X2
#H1: person incrreases social media ad popularity. 
#H2: instrument decreases social media ad popularity. 
#H3: There is synergy effect between person and instrument in increasing social media ad popularity. 

In [37]:
dummy_data2 = dummy_data.copy()

In [38]:
#sometimes, to avoid multicollinearity issue, we can reduce the value of brightness and saturation by their average before making interaction term
for col in ["YesFinalInstrument","yesPerson"]:
  avg = dummy_data2[col].mean()
  dummy_data2[col] = dummy_data2[col].apply(lambda x:x-avg)

In [39]:
#create the interaction term for brightness and saturation
dummy_data2["BxS"] = dummy_data2["YesFinalInstrument"] * dummy_data2["yesPerson"]

In [40]:
Y=dummy_data2['likeCount']
X=dummy_data2.drop(['likeCount'],axis=1).astype(float)
X=sm.add_constant(X)
model=sm.OLS(Y,X)
results=model.fit()

In [41]:
vif = variance_inflation_factor(X)
vif_store=vif.drop('const',axis=0)
vif_store.sort_values('vif',ascending=False)
df_coeffs = pd.DataFrame({"Coefficients": results.params, "p": results.pvalues, "vif": vif["vif"]})
if 'const' in df_coeffs.index:
    df_coeffs = df_coeffs.drop("const", axis=0)
df_coeffs_sorted = df_coeffs.reindex(df_coeffs["p"].sort_values(ascending=True).index)

In [42]:
df_coeffs_sorted

,Coefficients,p,vif
YesFinalInstrument,28.304890,3.075367e-09,1.024819
yesPerson,-14.929041,7.425110e-03,1.042779
BxS,-0.336473,9.782550e-01,1.051911


in this regression result, we can see the effect of brightness by itself, saturation by itself, and the combination of brightness and saturation on number of likes

#Research Question: Does combination of Brightness and Saturation can affect the number of likes? (Approach 2)
# Approach 2 - make HH, HL, LH, LL (For baseline, it is good to choose the variable with the smallest coeffient)
#H1: High brightness & High Satulation increases social media ad popularity. 
#H2: High brightness & Low Satulation decreases social media ad popularity. 
#H3: Low brightness and High Satulation increases social media ad popularity.

In [ ]:
dummy_data3 = dummy_data.copy()

In [ ]:
#convert into binary (0-1) using their mean as a cut off value
#Where 1 means high value, 0 mean low value
for col in ["brightness","saturation"]:
  avg = dummy_data3[col].mean()
  dummy_data3[col] = dummy_data3[col].apply(lambda x:1 if x>avg else 0)

In [ ]:
#create the interaction term for the combination between High-High, High-Low,Low-High,Low-Low
main_variable = ["brightness","saturation"]


In [ ]:
dummy_data3["High-High"] = dummy_data3[main_variable[0]] * dummy_data3[main_variable[1]] 

In [ ]:
dummy_data3["High-Low"] = dummy_data3[main_variable[0]] * (1 - dummy_data3[main_variable[1]] )

In [ ]:
dummy_data3["Low-High"] = (1-dummy_data3[main_variable[0]]) * dummy_data3[main_variable[1]]

In [ ]:
dummy_data3["Low-Low"] = (1-dummy_data3[main_variable[0]]) * (1 - dummy_data3[main_variable[1]] )

In [ ]:
#among 4 combination, drop 1 of them for regression as a baseline + original variable
Y=dummy_data3['likeCount']
X=dummy_data3.drop(['likeCount','Low-Low',"brightness","saturation"],axis=1).astype(float)
X=sm.add_constant(X)
model=sm.OLS(Y,X)
results=model.fit()

In [ ]:
vif = variance_inflation_factor(X)
vif_store=vif.drop('const',axis=0)
vif_store.sort_values('vif',ascending=False)
df_coeffs = pd.DataFrame({"Coefficients": results.params, "p": results.pvalues, "vif": vif["vif"]})
if 'const' in df_coeffs.index:
    df_coeffs = df_coeffs.drop("const", axis=0)
df_coeffs_sorted = df_coeffs.reindex(df_coeffs["p"].sort_values(ascending=True).index)

In [ ]:
df_coeffs_sorted

,Coefficients,p,vif
High-Low,100.0,0.454371,1.8
Low-High,-100.0,0.500000,1.6
High-High,-50.0,0.704833,1.6


 in this regression result, we can see the effect of 3 different pair of brightness and saturation on number of likes
